In [ ]:
import adagram
from allennlp.commands.elmo import ElmoEmbedder
import pandas as pd
from lxml import html
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from tqdm.notebook import tqdm
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.cluster import *
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')
stops = set(stopwords.words('russian'))

def normalize_pm(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

def tokenize(text):
    return token.tokenize(text)

## Адаграм

Word2Vec и многие другие векторные модели сопоставляют 1 вектор. Это значит, что у каждого слова в векторном пространстве только 1 значение. У многозначных слов векторы будут просто каким-то усреднением или обобщением всех его значений. 

В работе https://arxiv.org/pdf/1502.07257.pdf предлагается способ улучшить Skip Gram, так чтобы каждому слову сопоставлялось K различных векторов, так что каждый из них представляет какое-то из его значений. При этом сам параметр K задавать не нужно, модель сама находит нужное количество "значений" для каждого слова.

Изначально этот  подход реализован на julia, но есть реализация на питоне - https://github.com/lopuhin/python-adagram

In [ ]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

Посмотрим на значения каких-нибудь слов.

In [ ]:
vm.word_sense_probs('вечер')

Посмотрим какие слова близки к каждому из значений.

In [ ]:
vm.sense_neighbors('вечер', 0)

In [ ]:
vm.sense_neighbors('вечер', 1)

In [ ]:
vm.sense_neighbors('вечер', 2)

Можно посмотреть на все слова у которых есть хотя бы 2 устойчивых значения.

In [ ]:
ambiguous = []
for i, word in enumerate(vm.dictionary.id2word):
    probs = vm.word_sense_probs(word)
    if len(probs) > 1:
        ambiguous.append(word)
print(ambiguous[:50])

In [ ]:
means = vm.disambiguate('вечер', normalize("Ради любви родителей, ради того, чтобы они снова также танцевали в их гостиной, наслаждаясь милыми семейными"))

In [ ]:
vm.sense_neighbors('вечер', np.argmax(means))

In [ ]:
means = vm.disambiguate('вечер', normalize("абонемент № 19 \"Камерные \" включает в себя и концерт лауреата последнего Конкурса Чайковского"))

In [ ]:
vm.sense_neighbors('вечер', np.argmax(means))

## WSD / WSI
Разрешение семантической/лексической неоднозначности/омонимии

Проверим, насколько хорошо выбирается значение на данных с [соревнования Диалога](http://www.dialog-21.ru/evaluation/2018/disambiguation/) (переиспользую [baseline](https://github.com/nlpub/russe-wsi-kit) соревнования)

**NB!** Большая модель AdaGram для русского языка, которую мы используем, обучена на корпусе с нормализацией *mystem*. Так что немного модифицируем нашу функцию нормализации.

In [ ]:
mystem = Mystem()

def disambiguate(model, word, context):
    word, _ = lemmatized_context(word)
    probs = model.disambiguate(word, lemmatized_context(context))
    return 1 + probs.argmax()


def lemmatized_context(s):
    return [w.lower() for w in mystem.lemmatize(" ".join(tokenize(s)))]


In [ ]:
df = pd.read_csv('data/train.baseline-adagram.csv', sep='\t')

In [ ]:
pd.set_option('display.max_colwidth', 1000)

In [ ]:
df.head(5)

In [ ]:
df['predict_sense_id'] = [disambiguate(vm, word, context)
                          for word, context in tqdm(zip(df['word'], df['context']), total=len(df))]

In [ ]:
per_word = df.groupby('word').apply(lambda f: adjusted_rand_score(f['gold_sense_id'], f['predict_sense_id'])).to_frame('ARI')
per_word_ari = per_word['ARI']
print('Mean word ARI: %.4f' % np.mean(per_word_ari))

In [ ]:
per_word

В качестве метрики используется [Adjuster Rand Index](https://en.wikipedia.org/wiki/Rand_index), а [вот ссылка на документацию](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html).

## Contextualized embeddings
[ELMo](https://arxiv.org/pdf/1802.05365.pdf) — модель, которая позволяет получить не просто вектор слова W,
а _вектор слова W в контексте C_.
Что происходит?
Обучаем двунаправленную (bidirectional) языковую модель примерно так*:

<img src="https://jalammar.github.io/images/Bert-language-modeling.png" alt="elmo" width="400"/>

Но затем мы не просто берем какие-то представления отдельных слов, а сохраняем все веса и пропускаем каждое 
предложение для новой задачи через такую сетку с этими весами. Получаем вектора для всех слов в предложении из нескольких слоев!

\* картинка из [блога](https://jalammar.github.io/) чувака по имени Jay Allamar, кстати, очень доступные объяснения всяких NLP-штук с картинками

In [ ]:
# Немножко кода для загрузки модели
class Elmo:
    def __init__(self, path=""):
        if path:
            self.elmo = ElmoEmbedder(options_file=path + "/options.json", weight_file=path + "/model.hdf5")
        else:
            self.elmo = ElmoEmbedder()

    def get_elmo_vector(self, tokens, layer):
        vectors = self.elmo.embed_sentence(tokens)
        X = []
        for vector in vectors[layer]:
            X.append(vector)

        X = np.array(X)

        return X
    
    def get_word_vector(self, word, tokens, layer):
        vectors = self.elmo.embed_sentence(tokens)
        for v, t in zip(vectors[layer], tokens):
            if t == word:
                return v

In [ ]:
model = Elmo("196")

In [ ]:
sentence = "многочисленные укрепленные монастыри также не являлись замками как таковыми — это были крепости"
tokens = normalize(sentence)
v = model.get_elmo_vector(tokens, 0)
print(tokens)
print(v.shape)

In [ ]:
print(model.get_word_vector('замок', tokens, 2))

Попробуем сначала нарисовать, что получается (пропустим немного заранее заготовленной магии matplotlib и PCA)

In [ ]:
def dim_reduction(X, n):
    pca = PCA(n_components=n)
    print("size of X: {}".format(X.shape))
    results = pca.fit_transform(X)
    print("size of reduced X: {}".format(results.shape))

    for i, ratio in enumerate(pca.explained_variance_ratio_):
        print("Variance retained ratio of PCA-{}: {}".format(i+1, ratio))

    return results

In [ ]:
def plot(word, token_list, labels, reduced_X):
    fig, ax = plt.subplots()
    colors = ['ro', 'bo', 'yo', 'go', 'co']
    label_color = {}
    for i, l in enumerate(set(labels)):
        label_color[l] = colors[i]

    i = 0
    points = []
    for j, (tokens, l) in enumerate(zip(token_list, labels)):
        color = label_color[l[0]]
        for k, w in enumerate(tokens):
            if w == word:
                ax.plot(reduced_X[i, 0], reduced_X[i, 1], color)
                points.append((j, k, reduced_X[i, 0], reduced_X[i, 1]))
            i += 1

    for p in points:
        s = token_list[p[0]]
        text = ' '.join(s[min(0, p[1] - 5):min(p[1] + 5, len(s))])

        # bold the word of interest in the sentence
        text = text.replace(word, r"$\bf{" + word + "}$")

        plt.annotate(text, xy=p[2:])
    ax.set_xlabel("PCA 1")
    ax.set_ylabel("PCA 2")

In [ ]:
df = pd.read_csv('train.csv', sep='\t')

In [ ]:
df[df['word'] == 'замок']

In [ ]:
sentences_1 = df[df['word']=='замок'][df['gold_sense_id']=='1'].sample(4, random_state=5)
sentences_2 = df[df['word']=='замок'][df['gold_sense_id']=='2'].sample(4, random_state=5)

In [ ]:
sentences = list(sentences_1['context']) + list(sentences_2['context'])
labels = list(sentences_1['gold_sense_id']) + list(sentences_2['gold_sense_id'])

In [ ]:
sentences

In [ ]:
X = np.concatenate(
    [model.get_elmo_vector(tokens=normalize_pm(sentences[idx]), layer=2) for idx, _ in enumerate(sentences)], axis=0
)

In [ ]:
X.shape

In [ ]:
X_reduce = dim_reduction(X=X, n=2)

In [ ]:
plot('среда', [normalize_pm(s) for s in sentences],  labels, X_reduce)

Что можно сделать с этими векторами в целях WSD?
* классификатор
* кластеризация

Попробуем разные методы кластеризации.

In [ ]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [ ]:
ARI = []

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['context'].apply(normalize_pm)
    labels = grouped_df.get_group(key)['gold_sense_id'].to_list()
    X = []
    gold_labels = []
    
    for i, text in enumerate(texts):
        v = model.get_word_vector(key, text, 2)
        if v is not None:
            X.append(v)
            gold_labels.append(labels[i])

    if not X:
        continue
    cluster = AffinityPropagation(damping=0.9)
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1
    
    ARI.append(adjusted_rand_score(gold_labels, labels))
    
    print(key, '  ', adjusted_rand_score(gold_labels, labels))
print(np.mean(ARI))